# Example 4: Debugging

Debugging is half the coding process in software, and in PyRTL, it's no
different. PyRTL provides some additional challenges when it comes to
debugging as a problem may surface long after the error was made. Fortunately,
PyRTL comes with various features to help you find mistakes.

In [102]:
import random
import io
from pyrtl.rtllib import adders, multipliers
import pyrtl
pyrtl.reset_working_block()

In [103]:
random.seed(93729473)  # used to make random calls deterministic for this example

This example covers **debugging strategies** for PyRTL.  For general python debugging
we recommend healthy use of the "assert" statement, and use of "pdb" for
tracking down bugs.  However, PyRTL introduces some new complexities because
the place **where  functionality is defined (when you construct and operate
on PyRTL classes) is separate in time from where that functionality is executed
(i.e. during simulation)**.  Thus, sometimes it hard to track down where a wire
might have come from, or what exactly it is doing.

In this example specifically, we will be building a **circuit that adds up three values**.
However, instead of building an add function ourselves or using the
built-in "+" function in PyRTL, we will instead use the **Kogge-Stone adders
in RtlLib**, the standard library for PyRTL.

In [104]:
# building three inputs
in1, in2, in3 = (pyrtl.Input(8, "in" + str(x)) for x in range(1, 4))
out = pyrtl.Output(10, "out")

In [105]:
add1_out = adders.kogge_stone(in1, in2)
add2_out = adders.kogge_stone(add1_out, in2)
out <<= add2_out

The most basic way of debugging PyRTL is to **connect a value to an output wire**
and use the **simulation to trace the output**. A simple "print" statement doesn't work
because the values in the wires are not populated during *creation* time

If we want to check the result of the first addition, we can connect an output wire
to the result wire of the first adder

In [106]:
debug_out = pyrtl.Output(9, "debug_out")
debug_out <<= add1_out

Now **simulate the circuit.** Let's create some random inputs to feed our adder.

In [107]:
vals1 = [int(2**random.uniform(1, 8) - 2) for _ in range(20)]
vals2 = [int(2**random.uniform(1, 8) - 2) for _ in range(20)]
vals3 = [int(2**random.uniform(1, 8) - 2) for _ in range(20)]

In [108]:
sim_trace = pyrtl.SimulationTrace()
sim = pyrtl.Simulation(tracer=sim_trace)
for cycle in range(len(vals1)):
    sim.step({
        'in1': vals1[cycle],
        'in2': vals2[cycle],
        'in3': vals3[cycle]})

tmp3589/1W:
Wire Traceback, most recent call last 
  File "C:\Users\kmaho\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
   File "C:\Users\kmaho\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\tornado\ioloop.py", line 888, in start
    handler_func(fd_obj, events)
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\tornado\stack_context.py", line 277, in null

In order to get the result data, you do not need to print a waveform of the trace
You always have the option to just **pull the data out of the tracer directly**

In [109]:
print("---- Inputs and debug_out ----")
print("in1:       ", str(sim_trace.trace['in1']))
print("in2:       ", str(sim_trace.trace['in2']))
print("debug_out: ", str(sim_trace.trace['debug_out']))
print('\n')

---- Inputs and debug_out ----
in1:        [2, 14, 19, 8, 41, 12, 131, 101, 42, 0, 90, 117, 0, 13, 77, 4, 8, 35, 4, 61]
in2:        [18, 22, 53, 8, 2, 48, 60, 90, 50, 13, 138, 35, 8, 47, 4, 6, 66, 154, 13, 240]
debug_out:  [20, 36, 72, 16, 43, 60, 191, 191, 92, 13, 228, 152, 8, 60, 81, 10, 74, 189, 17, 301]




Below, I am using the ability to directly retrieve the trace data to
**verify the correctness of the first adder**

In [110]:
for i in range(len(vals1)):
    assert(sim_trace.trace['debug_out'][i] == sim_trace.trace['in1'][i] + sim_trace.trace['in2'][i])

## Probe

Now that we have built some stuff, let's clear it so we can try again in a
different way.  We can start by clearing all of the hardware from the current working
block.  The **working block is a global structure that keeps track of all the
hardware you have built thus far**.  A "reset" will **clear it so we can start fresh**.

In [111]:
pyrtl.reset_working_block()

In this example, we will be **multiplying two numbers** using *tree_multiplier()*
Again, create the two inputs and an output

In [112]:
print("---- Using Probes ----")
in1, in2 = (pyrtl.Input(8, "in" + str(x)) for x in range(1, 3))
out1, out2 = (pyrtl.Output(8, "out" + str(x)) for x in range(1, 3))

multout = multipliers.tree_multiplier(in1, in2)

#The following line will create a probe named "std_probe for later use, like an output.
pyrtl.probe(multout, 'std_probe')

---- Using Probes ----
Probe: std_probe Wire Traceback, most recent call last 
  File "C:\Users\kmaho\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
   File "C:\Users\kmaho\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\tornado\ioloop.py", line 888, in start
    handler_func(fd_obj, events)
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\tornado\stack_co

We could also do the same thing during assignment. The next command will
create a probe (named 'stdout_probe') that refers to multout (returns the wire multout).
This achieves virtually the same thing as 4 lines above, but it is done during assignment,
so we **skip a step by probing the wire before the multiplication.**
The probe returns multout, the original wire, and out will be assigned multout * 2

In [113]:
out1 <<= pyrtl.probe(multout, 'stdout_probe') * 2

Probe: stdout_probe Wire Traceback, most recent call last 
  File "C:\Users\kmaho\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
   File "C:\Users\kmaho\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\tornado\ioloop.py", line 888, in start
    handler_func(fd_obj, events)
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\tornado\stack_context.py", line 277,

#### Probe can also be used with other operations like this:

In [114]:
pyrtl.probe(multout + 32, 'adder_probe')

Probe: adder_probe Wire Traceback, most recent call last 
  File "C:\Users\kmaho\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
   File "C:\Users\kmaho\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\tornado\ioloop.py", line 888, in start
    handler_func(fd_obj, events)
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\tornado\stack_context.py", line 277, 

In [115]:
pyrtl.probe(multout[2:7], 'select_probe')

Probe: select_probe Wire Traceback, most recent call last 
  File "C:\Users\kmaho\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
   File "C:\Users\kmaho\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\tornado\ioloop.py", line 888, in start
    handler_func(fd_obj, events)
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\tornado\stack_context.py", line 277,

In [116]:
out2 <<= pyrtl.probe(multout)[2:16]  # notice probe names are not absolutely necessary

Probe: (Probe-5: tmp4213) Wire Traceback, most recent call last 
  File "C:\Users\kmaho\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
   File "C:\Users\kmaho\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\tornado\ioloop.py", line 888, in start
    handler_func(fd_obj, events)
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\tornado\stack_context.py", lin

As one can see, probe **can be used on any wire any time**,
such as before or during its operation, assignment, etc.
#### Now on to the simulation...
For variation, we'll **recreate the random inputs**:

In [117]:
vals1 = [int(2**random.uniform(1, 8) - 2) for _ in range(10)]
vals2 = [int(2**random.uniform(1, 8) - 2) for _ in range(10)]

In [118]:
sim_trace = pyrtl.SimulationTrace()
sim = pyrtl.Simulation(tracer=sim_trace)
for cycle in range(len(vals1)):
    sim.step({
        'in1': vals1[cycle],
        'in2': vals2[cycle]})

tmp4104/1W:
Wire Traceback, most recent call last 
  File "C:\Users\kmaho\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
   File "C:\Users\kmaho\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\tornado\ioloop.py", line 888, in start
    handler_func(fd_obj, events)
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\tornado\stack_context.py", line 277, in null

Now we will **show the values of the inputs and probes**
and look at that, we didn't need to make any outputs!
(although we did, to demonstrate the power and convenience of probes)

In [119]:
sim_trace.render_trace()
sim_trace.print_trace()

AttributeError: 'str' object has no attribute 'name'

Say we wanted to have gotten **more information about
one of those probes** above at declaration.
We could have used *pyrtl.set_debug_mode()* before their creation, like so:

In [120]:
print("--- Probe w/ debugging: ---")
pyrtl.set_debug_mode()
pyrtl.probe(multout - 16, 'debugsubtr_probe)')
pyrtl.set_debug_mode(debug=False)

--- Probe w/ debugging: ---
Probe: debugsubtr_probe) Wire Traceback, most recent call last 
  File "C:\Users\kmaho\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
   File "C:\Users\kmaho\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\tornado\ioloop.py", line 888, in start
    handler_func(fd_obj, events)
   File "C:\Users\kmaho\Anaconda3\lib\site-packages\tor

## WireVector Stack Trace

Another case that might arise is that a certain wire is causing an error to occur
in your program. WireVector Stack Traces allow you to find out more about **where a particular
WireVector was made in your code**. With this enabled the WireVector will
store exactly were it was created, which should help with issues where
there is a problem with an identified wire.

Like above, just add the following line before the relevant WireVector
might be made or at the beginning of the program.

In [ ]:
pyrtl.set_debug_mode()

print("---- Stack Trace ----")
test_out = pyrtl.Output(9, "test_out")
test_out <<= adders.kogge_stone(in1, in2)

#### Now to retrieve information:

In [ ]:
wire_trace = test_out.init_call_stack

This data is generated using the __*traceback.format_stack()*__ call from the Python
standard library's Traceback module (look at the Python standard library docs for
details on the function). Therefore, the **stack traces are stored as a list with the
outermost call first**.

In [ ]:
for frame in wire_trace:
    print(frame)

## Storage of Additional Debug Data

__*WARNING: the debug information generated by the following two processes are
not guaranteed to be preserved when functions (eg. pyrtl.synthesize() ) are
done over the block.*__

However, if the stack trace does not give you enough information about the
WireVector, you can also **embed additional information into the wire** itself.

Two ways of doing so is either through **manipulating the name of the
WireVector**, or by **adding your own custom metadata** to the WireVector.

So far, each input and output WireVector have been given their own names, but
normal WireVectors can also be given names by **supplying the name argument to
the constructor**

In [121]:
dummy_wv = pyrtl.WireVector(1, name="blah")

Also, because of the flexible nature of Python, you can also add **custom
properties to the WireVector.**

In [122]:
dummy_wv.my_custom_property_name = "John Clow is great"
dummy_wv.custom_value_028493 = 13

In [123]:
# removing the WireVector from the block to prevent problems with the rest of
# this example
pyrtl.working_block().remove_wirevector(dummy_wv)

### Trivial Graph Format

Finally, there is a handy way to **view your hardware creations as a graph**.
The function *output_to_trivialgraph* will render your hardware a formal that
you can then open with the free software *"yEd"*
(http://en.wikipedia.org/wiki/YEd). There are options under the
*"hierarchical"* rendering to draw something that looks quite like a circuit.

In [124]:
pyrtl.working_block().sanity_check()
pyrtl.passes._remove_unused_wires(pyrtl.working_block())  # so that trivial_graph() will work

In [125]:
print("--- Trivial Graph Format  ---")
with io.StringIO() as tgf:
    pyrtl.output_to_trivialgraph(tgf)
    print(tgf.getvalue())

--- Trivial Graph Format  ---
0 ^
1 &
2 &
3 ^
4 ^
5 ^
6 |
7 |
8 |
9 &
10 &
11 &
12 s(1,)
13 s(0,)
14 &
15 &
16 s(1,)
17 ^
18 &
19 &
20 &
21 |
22 |
23 &
24 |
25 |
26 |
27 ^
28 &
29 &
30 ^
31 |
32 |
33 |
34 &
35 &
36 &
37 &
38 ^
39 &
40 &
41 s(4,)
42 &
43 &
44 ^
45 &
46 ^
47 &
48 s(7,)
49 &
50 ^
51 &
52 &
53 &
54 s(0,)
55 &
56 &
57 |
58 &
59 &
60 &
61 &
62 w
63 ^
64 ^
65 &
66 &
67 &
68 s(5,)
69 s(3,)
70 |
71 |
72 &
73 &
74 |
75 &
76 ^
77 ^
78 &
79 &
80 &
81 |
82 |
83 |
84 &
85 |
86 s(0,)
87 &
88 ^
89 &
90 |
91 |
92 &
93 &
94 &
95 +
96 ^
97 s(2,)
98 ^
99 &
100 s(4,)
101 ^
102 ^
103 &
104 &
105 &
106 &
107 &
108 ^
109 &
110 &
111 &
112 ^
113 |
114 &
115 |
116 ^
117 |
118 &
119 &
120 s(1,)
121 &
122 &
123 ^
124 &
125 |
126 ^
127 |
128 s(2,)
129 s(2,)
130 &
131 |
132 |
133 |
134 s(7,)
135 s(2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15)
136 &
137 ^
138 |
139 c
140 ^
141 &
142 s(2,)
143 s(1,)
144 &
145 &
146 s(0,)
147 c
148 s(2,)
149 |
150 s(1,)
151 |
152 &
153 &
154 ^
155 |
156 |
157 |
158 